In [15]:
import numpy as np
import datetime as dt

In [24]:
data = np.loadtxt('data.txt', skiprows=2)
# date = dt.datetime(data[0], data[1], data[2], data[4].strip('.')[0], minute)
print(data[4])



[2.023e+03 1.200e+01 1.000e+00 3.350e+02 1.200e+01 8.000e+00 5.600e+01
 4.400e+01 9.999e-38 8.478e+18 2.273e+18 5.219e-04 2.256e+02 1.120e+03
 5.646e+13 1.013e+17 9.999e-38 9.999e-38 1.666e+02 1.666e+02 4.920e+01
 6.700e+01 1.110e+02 2.700e+01 2.200e+01 7.500e+00 4.400e+00]
